# IMDB 电影评论情感分类数据集

数据集来自 IMDB 的 25,000 条电影评论，以情绪（正面/负面）标记。评论已经过预处理，并编码为词索引（整数）的序列表示。为了方便起见，将词按数据集中出现的频率进行索引，例如整数 3 编码数据中第三个最频繁的词。这允许快速筛选操作，例如：「只考虑前 10,000 个最常用的词，但排除前 20 个最常见的词」。

作为惯例，0 不代表特定的单词，而是被用于编码任何未知单词。

In [1]:
from keras.datasets import imdb

(X_train, y_train), (X_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=10000, # 限制只考虑10000个常用词
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)
# 测试模型时可以使用小样本集，调参数时推荐使用完整数据集
# X_train, y_train, X_test, y_test = X_train[:5000], y_train[:5000], X_test[:5000], y_test[:5000]

print("X训练集特征数据的部分示例为：\n", X_train[:1])
print("y训练集标签数据的部分示例为：\n", y_train[:5])

Using TensorFlow backend.
/Users/slyfox/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/slyfox/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/slyfox/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/slyfox/opt/anaconda3/lib/python3.7

X训练集特征数据的部分示例为：
 [list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])]
y训练集标签数据的部分示例为：
 [1 0 0 1 0]


## 数据预处理
不能直接将list类型的数据送到神经网络中训练，必须将list类型转换为tensor张量类型。有两种转换方式：

1、将列表进行multi-hot编码，通过词袋模型的方法将每一个文档转化为0、 1向量。然后使用全连接层构建分类器。

2、填充列表使每个列表长度都相同，然后将每个词分别转化为语义向量，形状为(samples, max_word_amount, word_vector_dim),使用循环神经网络对每一个时间步的输入进行时序编码，构建模型。

In [2]:
import numpy as np

# 通过词袋模型将每个评论转化为长度为10000的定长向量
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension)) #数据集长度，每个评论维度10000
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1 # one-hot
    return results

X_train = vectorize_sequences(X_train)
X_test = vectorize_sequences(X_test)

print("转化为词袋模型的X特征集示例为：", X_train[:1])

y_train = np.asarray(y_train).astype('float32') # 向量化标签数据
y_test = np.asarray(y_test).astype('float32')

转化为词袋模型的X特征集示例为： [[0. 1. 1. ... 0. 0. 0.]]


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

max_features = 10000  # 词仅采用了10000以内的整数作为编码

model = Sequential()
model.add(Dense(16, activation='relu',input_shape=(10000,)))
# 如果模型的参数太多，而训练样本又太少，训练出来的模型很容易产生过拟合的现象。dropout让模型不会太依赖某些局部的特征
model.add(Dropout(0.5))  
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model_history = model.fit(X_train, y_train, batch_size=32, epochs=20)
score = model.evaluate(X_test, y_test, batch_size=16)

print("测试集上的测试误差及准确率为：", score)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
25000/25000 [==============================] - 4s 141us/step - loss: 0.4853 - accuracy: 0.7715
Epoch 2/20
25000/25000 [==============================] - 3s 119us/step - loss: 0.3631 - accuracy: 0.8632
Epoch 3/20
25000/25000 [==============================] - 2s 92us/step - loss: 0.3394 - accuracy: 0.8791
Epoch 4/20
25000/25000 [==============================] - 2s 94us/step - loss: 0.3218 - accuracy: 0.8862
Epoch 5/20
25000/25000 [==============================] - 2s 99us/step - loss: 0.3056 - accuracy: 0.8942
Epoch 6/20
25000/25000 [==============================] - 2s 92us/step - loss: 0.3109 - accuracy: 0.8934
Epoch 7/20
25000/25000 [==============================] - 2s 98us/step - loss: 0.3011 - accuracy: 0.8952
Epoch 8/20
25000/25000 [==============================] - 2s 95us/step - loss: 0.3033 - accuracy: 0.8960
Epoch 9/20
25000/25000 [==============

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                160016    
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


## 扩展作业
1、有兴趣的同学可以思考一下，如何将词向量模型应用到神经网络中。

2、以Keras自带的 路透社新闻主题分类 数据集复现实验并调整网络架构以及模型参数。学习率及优化器的调整可参考学习脚本9_Instances_of_sequential_model中优化器的定义。

3、在训练集上进行拆分，添加验证集以验证模型在训练过程中的表现。